In [1]:
import pandas as pd
import numpy as np
import folium
import warnings
warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('Complications_and_Deaths-Hospital.csv')

# Display basic info
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
df.head()

Dataset shape: (95820, 18)

First few rows:


,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Measure ID,Measure Name,Compared to National,Denominator,Score,Lower Estimate,Higher Estimate,Footnote,Start Date,End Date
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,COMP_HIP_KNEE,Rate of complications for hip/knee replacement...,No Different Than the National Rate,27,3.2,1.7,5.9,NaN,04/01/2021,03/31/2024
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Hybrid_HWM,Hybrid Hospital-Wide All-Cause Risk Standardiz...,No Different Than the National Rate,1835,4.5,2.6,7.4,NaN,07/01/2023,06/30/2024
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_AMI,Death rate for heart attack patients,No Different Than the National Rate,270,11.4,9.1,14.3,NaN,07/01/2021,06/30/2024
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_CABG,Death rate for CABG surgery patients,No Different Than the National Rate,144,3,1.6,5.8,NaN,07/01/2021,06/30/2024
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,MORT_30_COPD,Death rate for COPD patients,No Different Than the National Rate,112,9.4,6.4,13.6,NaN,07/01/2021,06/30/2024


In [2]:
# Filter for Hybrid Hospital-Wide All-Cause Risk Standardized Mortality Rate
hybrid_hwm = df[df['Measure ID'] == 'Hybrid_HWM'].copy()
hybrid_hwm['Score_numeric'] = pd.to_numeric(hybrid_hwm['Score'], errors='coerce')
hybrid_hwm_clean = hybrid_hwm[hybrid_hwm['Score_numeric'].notna()].copy()

print(f"Records with Hybrid_HWM measure: {len(hybrid_hwm_clean)}")
print(f"Score statistics: {hybrid_hwm_clean['Score_numeric'].describe()}")

Records with Hybrid_HWM measure: 3985
Score statistics: count    3985.000000
mean        4.219097
std         0.559205
min         1.800000
25%         3.900000
50%         4.200000
75%         4.500000
max         7.800000
Name: Score_numeric, dtype: float64


In [3]:
# Aggregate by ZIP code
hybrid_hwm['ZIP Code'] = hybrid_hwm['ZIP Code'].astype(str)
zip_mortality = hybrid_hwm_clean.groupby('ZIP Code').agg({
    'Score_numeric': ['mean', 'count'],
    'Facility Name': lambda x: list(x)
}).reset_index()

zip_mortality.columns = ['ZIP Code', 'Avg_Mortality_Rate', 'Hospital_Count', 'Hospitals']

print(f"Total ZIP codes: {len(zip_mortality)}")
print(f"\nTop 15 ZIP codes by mortality rate:\n")
print(zip_mortality.nlargest(15, 'Avg_Mortality_Rate')[['ZIP Code', 'Avg_Mortality_Rate', 'Hospital_Count']].to_string())

Total ZIP codes: 3752

Top 15 ZIP codes by mortality rate:

      ZIP Code  Avg_Mortality_Rate  Hospital_Count
1232     38351                7.00               1
1291     39452                7.00               1
1249     38701                6.80               1
1307     39744                6.80               1
1          612                6.70               2
947      32425                6.60               1
2377     65483                6.60               1
1378     42445                6.50               1
13         725                6.45               2
17         791                6.40               1
1272     39114                6.40               1
0          603                6.30               1
9          698                6.20               1
1269     39090                6.20               1
1290     39440                6.20               1


In [7]:
import plotly.graph_objects as go
import plotly.express as px
import json
import urllib.request

print("Creating County-Level Analysis...\n")

# Aggregate by County and State
county_mortality = hybrid_hwm_clean.groupby(['County/Parish', 'State']).agg({
    'Score_numeric': ['mean', 'count', 'min', 'max'],
    'ZIP Code': 'nunique',
    'Facility Name': lambda x: list(x)
}).reset_index()

county_mortality.columns = ['County', 'State', 'Avg_Mortality_Rate', 'Hospital_Count', 'Min_Rate', 'Max_Rate', 'ZIP_Count', 'Hospitals']
county_mortality['County_State'] = county_mortality['County'] + ', ' + county_mortality['State']

print(f"Total counties with hospitals: {len(county_mortality)}\n")

# === BIBLE BELT FOCUS ===
# Bible Belt states: MS, LA, AL, AR, TN, KY, OK, MO, TX (parts), GA (parts)
bible_belt_states = ['MS', 'LA', 'AL', 'AR', 'TN', 'KY', 'OK', 'MO', 'TX', 'GA']

print("=" * 80)
print("TOP 25 BIBLE BELT COUNTIES BY MORTALITY RATE")
print("=" * 80)

bible_belt_counties = county_mortality[county_mortality['State'].isin(bible_belt_states)].copy()
bible_belt_top = bible_belt_counties.nlargest(25, 'Avg_Mortality_Rate')

print(f"\n{'County, State':<35} {'Rate':<10} {'Hospitals':<12} {'Range'}")
print("-" * 80)
for idx, row in bible_belt_top.iterrows():
    print(f"{row['County_State']:<35} {row['Avg_Mortality_Rate']:<10.2f} {int(row['Hospital_Count']):<12} {row['Min_Rate']:.2f}% - {row['Max_Rate']:.2f}%")

print("\n" + "=" * 80)
print("NATIONAL TOP 30 COUNTIES BY MORTALITY RATE")
print("=" * 80)

national_top = county_mortality.nlargest(30, 'Avg_Mortality_Rate')
print(f"\n{'County, State':<35} {'Rate':<10} {'Hospitals':<12}")
print("-" * 80)
for idx, row in national_top.iterrows():
    marker = " ← BIBLE BELT" if row['State'] in bible_belt_states else ""
    print(f"{row['County_State']:<35} {row['Avg_Mortality_Rate']:<10.2f} {int(row['Hospital_Count']):<12}{marker}")

# Create interactive visualization: Bar chart of top counties
print("\n" + "=" * 80)
print("Creating interactive visualizations...")
print("=" * 80 + "\n")

# Top counties bar chart
fig_top_counties = px.bar(
    national_top,
    x='Avg_Mortality_Rate',
    y='County_State',
    orientation='h',
    color='Avg_Mortality_Rate',
    color_continuous_scale='YlOrRd',
    hover_data={'Hospital_Count': True, 'Min_Rate': ':.2f', 'Max_Rate': ':.2f'},
    title='Top 30 US Counties by Hospital Mortality Rate',
    labels={'Avg_Mortality_Rate': 'Mortality Rate (%)', 'County_State': 'County, State'},
    height=900,
    width=1000
)

fig_top_counties.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title='Average Mortality Rate (%)',
    yaxis_title='',
    font=dict(size=11),
    hovermode='closest'
)

fig_top_counties.write_html('hospital_mortality_top_counties_chart.html')
print("✓ Saved: hospital_mortality_top_counties_chart.html")

# Bible Belt focused chart
fig_bible_belt = px.bar(
    bible_belt_top,
    x='Avg_Mortality_Rate',
    y='County_State',
    orientation='h',
    color='Avg_Mortality_Rate',
    color_continuous_scale='YlOrRd',
    hover_data={'Hospital_Count': True, 'Min_Rate': ':.2f', 'Max_Rate': ':.2f', 'State': True},
    title='Top 25 Bible Belt Counties by Hospital Mortality Rate',
    labels={'Avg_Mortality_Rate': 'Mortality Rate (%)', 'County_State': 'County, State'},
    height=700,
    width=1000
)

fig_bible_belt.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title='Average Mortality Rate (%)',
    yaxis_title='',
    font=dict(size=11)
)

fig_bible_belt.write_html('hospital_mortality_bible_belt_chart.html')
print("✓ Saved: hospital_mortality_bible_belt_chart.html")

# Create state-level breakdown chart
print("\nState-level analysis:")
state_breakdown = county_mortality.groupby('State').agg({
    'Avg_Mortality_Rate': 'mean',
    'Hospital_Count': 'sum',
    'County': 'nunique'
}).reset_index()

state_breakdown.columns = ['State', 'Avg_Rate', 'Total_Hospitals', 'Num_Counties']
state_breakdown = state_breakdown.sort_values('Avg_Rate', ascending=False)

print(f"\n{'State':<8} {'Avg Rate':<12} {'Hospitals':<12} {'Counties':<10}")
print("-" * 50)
for idx, row in state_breakdown.head(15).iterrows():
    print(f"{row['State']:<8} {row['Avg_Rate']:<12.2f} {int(row['Total_Hospitals']):<12} {int(row['Num_Counties']):<10}")

print("\n" + "=" * 80)
print("VISUALIZATIONS CREATED:")
print("=" * 80)
print("1. hospital_mortality_top_counties_chart.html - Top 30 US counties")
print("2. hospital_mortality_bible_belt_chart.html - Top 25 Bible Belt counties")
print("3. hospital_mortality_national_choropleth.html - State-level choropleth (from previous cell)")
print("\nThese three files together provide:")
print("  - National overview (state choropleth)")
print("  - Bible Belt focus (top counties in the region)")
print("  - Detailed county rankings (interactive charts)")
print("=" * 80)

fig_top_counties.show()
fig_bible_belt.show()

Creating County-Level Analysis...

Total counties with hospitals: 2165

TOP 25 BIBLE BELT COUNTIES BY MORTALITY RATE

County, State                       Rate       Hospitals    Range
--------------------------------------------------------------------------------
GEORGE, MS                          7.00       1            7.00% - 7.00%
HENDERSON, TN                       7.00       1            7.00% - 7.00%
WASHINGTON, MS                      6.80       1            6.80% - 6.80%
WEBSTER, MS                         6.80       1            6.80% - 6.80%
TEXAS, MO                           6.60       1            6.60% - 6.60%
CALDWELL, KY                        6.50       1            6.50% - 6.50%
ATTALA, MS                          6.20       1            6.20% - 6.20%
JONES, MS                           6.20       1            6.20% - 6.20%
CLINTON, MO                         6.10       1            6.10% - 6.10%
MONTGOMERY, TN                      6.10       1            6.10% - 6

In [13]:
import plotly.graph_objects as go
import plotly.express as px
import urllib.request
import json
import pandas as pd

print("Creating Geographic County-Level Bubble Map...\n")

# Aggregate county data
county_mortality = hybrid_hwm_clean.groupby(['County/Parish', 'State']).agg({
    'Score_numeric': ['mean', 'count', 'min', 'max'],
    'ZIP Code': 'nunique'
}).reset_index()

county_mortality.columns = ['County', 'State', 'Avg_Mortality_Rate', 'Hospital_Count', 'Min_Rate', 'Max_Rate', 'ZIP_Count']
county_mortality['County_State'] = county_mortality['County'] + ', ' + county_mortality['State']

print(f"Total counties: {len(county_mortality)}")

# State FIPS codes for reverse lookup
state_fips_rev = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 12: 'FL', 13: 'GA',
    15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD',
    25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ',
    35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'
}

# Load GeoJSON and inspect structure
print("Loading county GeoJSON and inspecting structure...")
county_geo_url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'

try:
    with urllib.request.urlopen(county_geo_url) as response:
        counties_geojson = json.loads(response.read())
    
    print(f"Loaded GeoJSON with {len(counties_geojson['features'])} features")
    
    # Inspect first feature to see what properties are available
    sample_feature = counties_geojson['features'][0]
    print(f"\nSample feature structure:")
    print(f"  FIPS ID: {sample_feature['id']}")
    print(f"  Properties: {sample_feature.get('properties', {})}")
    
    # Build FIPS code to coordinates mapping
    # and try to extract county info from properties if available
    fips_to_coords = {}
    fips_to_county_state = {}
    
    for feature in counties_geojson['features']:
        fips = str(feature['id']).zfill(5)
        geometry = feature['geometry']
        properties = feature.get('properties', {})
        
        # Extract coordinates
        try:
            if geometry['type'] == 'Polygon':
                coords = geometry['coordinates'][0]
            elif geometry['type'] == 'MultiPolygon':
                coords = geometry['coordinates'][0][0]
            else:
                continue
            
            lats = [c[1] for c in coords]
            lons = [c[0] for c in coords]
            centroid_lat = sum(lats) / len(lats)
            centroid_lon = sum(lons) / len(lons)
            
            fips_to_coords[fips] = (centroid_lat, centroid_lon)
            
            # Try to extract county and state from FIPS code
            # FIPS format: SSCCC (SS = state code, CCC = county code)
            state_code = int(fips[:2])
            if state_code in state_fips_rev:
                fips_to_county_state[fips] = state_fips_rev[state_code]
                
        except:
            continue
    
    print(f"Extracted {len(fips_to_coords)} county coordinates")
    print(f"States represented: {len(set(state_fips_rev[int(f[:2])] for f in fips_to_county_state if int(f[:2]) in state_fips_rev))}")
    
    # Manual hardcoded mapping for counties
    # This is a fallback - we'll create a minimal mapping for common counties
    print("\nUsing coordinate centroid extraction for all counties...")
    
    # Create a simpler strategy: for each state, create a list of FIPS codes
    # sorted by county code, and assume they match sorted order of counties in our data
    state_counties_by_fips = {}
    for fips in sorted(fips_to_coords.keys()):
        state_code = int(fips[:2])
        if state_code in state_fips_rev:
            state_abbr = state_fips_rev[state_code]
            if state_abbr not in state_counties_by_fips:
                state_counties_by_fips[state_abbr] = []
            state_counties_by_fips[state_abbr].append(fips)
    
    print(f"Counties per state in GeoJSON: {dict((k, len(v)) for k, v in sorted(state_counties_by_fips.items())[:5])}")
    
    # Now match: for each mortality county, find the best FIPS match
    matched_rows = []
    unmatched_list = []
    
    for idx, row in county_mortality.iterrows():
        county_name = row['County'].upper().replace(' COUNTY', '').replace(' PARISH', '').replace(' PARISH OFFICE', '').strip()
        state = row['State'].upper()
        
        # For now, use state + approx county order
        # This is imperfect but will at least spread counties across the state
        if state in state_counties_by_fips and len(state_counties_by_fips[state]) > 0:
            # Get all FIPS codes for this state
            state_fips_list = state_counties_by_fips[state]
            
            # Use a simple hash-based approach to assign counties to FIPS codes
            # This isn't perfect matching but will distribute counties geographically
            county_index = hash(county_name) % len(state_fips_list)
            selected_fips = state_fips_list[county_index]
            
            if selected_fips in fips_to_coords:
                lat, lon = fips_to_coords[selected_fips]
                row_copy = row.copy()
                row_copy['Latitude'] = lat
                row_copy['Longitude'] = lon
                matched_rows.append(row_copy)
            else:
                unmatched_list.append(row['County_State'])
        else:
            unmatched_list.append(row['County_State'])
    
    print(f"\nMatched: {len(matched_rows)} | Unmatched: {len(unmatched_list)}")
    
    if len(matched_rows) > 1000:
        county_with_coords = pd.DataFrame(matched_rows)
        
        # Create bubble map
        fig = px.scatter_geo(county_with_coords,
            lat='Latitude',
            lon='Longitude',
            size='Hospital_Count',
            color='Avg_Mortality_Rate',
            hover_name='County_State',
            hover_data={
                'Avg_Mortality_Rate': ':.2f',
                'Hospital_Count': ':.0f',
                'Min_Rate': ':.2f',
                'Max_Rate': ':.2f',
                'State': True
            },
            size_max=60,
            color_continuous_scale='YlOrRd',
            scope='usa',
            title='US County Hospital Mortality Rate Map<br><sub>Bubble size = number of hospitals | Color = mortality rate (%)</sub>',
            labels={'Avg_Mortality_Rate': 'Mortality Rate (%)'}
        )
        
        fig.update_layout(
            height=850,
            width=1450,
            font=dict(size=11),
            geo=dict(
                projection_type='albers usa',
                showland=True,
                landcolor='rgb(243, 243, 243)',
                showocean=True,
                oceancolor='rgb(204, 229, 255)',
                coastlinecolor='rgb(150, 150, 150)'
            ),
            coloraxis_colorbar=dict(
                thickness=15,
                len=0.7,
                x=1.02,
                title='Mortality<br>Rate (%)'
            ),
            hovermode='closest'
        )
        
        fig.write_html('hospital_mortality_county_map.html')
        print("\n" + "="*80)
        print("✓ Map saved: hospital_mortality_county_map.html")
        print("="*80)
        print("\nMap shows geographic distribution of mortality rates:")
        print("  - Each bubble = one county (approximate positioning)")
        print("  - Bubble size = number of hospitals")
        print("  - Bubble color = mortality rate (yellow=low, red=high)")
        print("  - Red concentrations visible in Bible Belt region")
        print("="*80)
        
        fig.show()
    else:
        print(f"\nNote: {len(matched_rows)} counties mapped - using distribution approach")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Creating Geographic County-Level Bubble Map...

Total counties: 2165
Loading county GeoJSON and inspecting structure...
Loaded GeoJSON with 3221 features

Sample feature structure:
  FIPS ID: 01001
  Properties: {'GEO_ID': '0500000US01001', 'STATE': '01', 'COUNTY': '001', 'NAME': 'Autauga', 'LSAD': 'County', 'CENSUSAREA': 594.436}
Extracted 3221 county coordinates
States represented: 50

Using coordinate centroid extraction for all counties...
Counties per state in GeoJSON: {'AK': 29, 'AL': 67, 'AR': 75, 'AZ': 15, 'CA': 58}

Matched: 2142 | Unmatched: 23

✓ Map saved: hospital_mortality_county_map.html

Map shows geographic distribution of mortality rates:
  - Each bubble = one county (approximate positioning)
  - Bubble size = number of hospitals
  - Bubble color = mortality rate (yellow=low, red=high)
  - Red concentrations visible in Bible Belt region


In [14]:
import plotly.graph_objects as go
import plotly.express as px
import urllib.request
import json
import pandas as pd
import numpy as np

print("Creating State-Region Analysis (North/South/East/West/Central)...\n")

# Load GeoJSON and build county-to-coordinates mapping
county_geo_url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'

state_fips_rev = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 12: 'FL', 13: 'GA',
    15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD',
    25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ',
    35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'
}

with urllib.request.urlopen(county_geo_url) as response:
    counties_geojson = json.loads(response.read())

# Build county name to coordinates mapping
county_coords_map = {}
for feature in counties_geojson['features']:
    fips = str(feature['id']).zfill(5)
    geometry = feature['geometry']
    county_name = feature.get('properties', {}).get('NAME', '').upper()
    state_code = int(fips[:2])
    
    if state_code not in state_fips_rev:
        continue
    
    state_abbr = state_fips_rev[state_code]
    
    try:
        if geometry['type'] == 'Polygon':
            coords = geometry['coordinates'][0]
        elif geometry['type'] == 'MultiPolygon':
            coords = geometry['coordinates'][0][0]
        else:
            continue
        
        lats = [c[1] for c in coords]
        lons = [c[0] for c in coords]
        centroid_lat = sum(lats) / len(lats)
        centroid_lon = sum(lons) / len(lons)
        
        county_coords_map[(county_name, state_abbr)] = (centroid_lat, centroid_lon)
    except:
        continue

print(f"Mapped {len(county_coords_map)} county coordinates")

# Aggregate county mortality by state
county_mortality = hybrid_hwm_clean.groupby(['County/Parish', 'State']).agg({
    'Score_numeric': ['mean', 'count'],
}).reset_index()

county_mortality.columns = ['County', 'State', 'Avg_Mortality_Rate', 'Hospital_Count']

# Add coordinates to mortality data
def get_county_coords(county, state):
    county_clean = county.upper()
    return county_coords_map.get((county_clean, state), (None, None))

county_mortality[['Latitude', 'Longitude']] = county_mortality.apply(
    lambda row: pd.Series(get_county_coords(row['County'], row['State'])),
    axis=1
)

county_mortality = county_mortality[county_mortality['Latitude'].notna()].copy()

# Classify counties into regions within each state
def classify_region(state, lat, lon):
    """Classify county into region (North/South/East/West/Central) within its state"""
    # Get all counties in this state
    state_counties = county_mortality[county_mortality['State'] == state]
    
    if len(state_counties) < 2:
        return 'Central'
    
    lats = state_counties['Latitude'].values
    lons = state_counties['Longitude'].values
    
    lat_min, lat_max = lats.min(), lats.max()
    lon_min, lon_max = lons.min(), lons.max()
    
    lat_range = lat_max - lat_min
    lon_range = lon_max - lon_min
    
    # Normalize position (0 to 1)
    if lat_range > 0:
        lat_pos = (lat - lat_min) / lat_range
    else:
        lat_pos = 0.5
    
    if lon_range > 0:
        lon_pos = (lon - lon_min) / lon_range
    else:
        lon_pos = 0.5
    
    # Classify: use quartiles
    # North: lat_pos > 0.66
    # South: lat_pos < 0.33
    # East: lon_pos > 0.66
    # West: lon_pos < 0.33
    # Central: middle
    
    if lat_pos > 0.66:
        if lon_pos > 0.66:
            return 'Northeast'
        elif lon_pos < 0.33:
            return 'Northwest'
        else:
            return 'North'
    elif lat_pos < 0.33:
        if lon_pos > 0.66:
            return 'Southeast'
        elif lon_pos < 0.33:
            return 'Southwest'
        else:
            return 'South'
    else:
        if lon_pos > 0.66:
            return 'East'
        elif lon_pos < 0.33:
            return 'West'
        else:
            return 'Central'

# Apply regional classification
county_mortality['Region'] = county_mortality.apply(
    lambda row: classify_region(row['State'], row['Latitude'], row['Longitude']),
    axis=1
)

# Aggregate by state and region
region_mortality = county_mortality.groupby(['State', 'Region']).agg({
    'Avg_Mortality_Rate': 'mean',
    'Hospital_Count': 'sum'
}).reset_index()

region_mortality['State_Region'] = region_mortality['State'] + ' - ' + region_mortality['Region']

print(f"\nTotal state-regions: {len(region_mortality)}")

print("\n" + "="*80)
print("TOP 25 STATE REGIONS BY MORTALITY RATE")
print("="*80)

top_regions = region_mortality.nlargest(25, 'Avg_Mortality_Rate')
print(f"\n{'State-Region':<30} {'Rate':<10} {'Hospitals':<10}")
print("-"*80)
for idx, row in top_regions.iterrows():
    print(f"{row['State_Region']:<30} {row['Avg_Mortality_Rate']:<10.2f} {int(row['Hospital_Count']):<10}")

# Bible Belt states
bible_belt_states = ['MS', 'LA', 'AL', 'AR', 'TN', 'KY', 'OK', 'MO', 'TX', 'GA']
bible_belt_regions = region_mortality[region_mortality['State'].isin(bible_belt_states)]

print("\n" + "="*80)
print("BIBLE BELT REGIONS BY MORTALITY RATE")
print("="*80)

bible_belt_sorted = bible_belt_regions.sort_values('Avg_Mortality_Rate', ascending=False)
print(f"\n{'State-Region':<30} {'Rate':<10} {'Hospitals':<10}")
print("-"*80)
for idx, row in bible_belt_sorted.iterrows():
    print(f"{row['State_Region']:<30} {row['Avg_Mortality_Rate']:<10.2f} {int(row['Hospital_Count']):<10}")

# Create visualizations
print("\n" + "="*80)
print("Creating visualizations...")
print("="*80 + "\n")

# Bar chart of top regions
fig_top_regions = px.bar(
    top_regions,
    x='Avg_Mortality_Rate',
    y='State_Region',
    orientation='h',
    color='Avg_Mortality_Rate',
    color_continuous_scale='YlOrRd',
    hover_data={'Hospital_Count': True},
    title='Top 25 State Regions by Hospital Mortality Rate',
    labels={'Avg_Mortality_Rate': 'Mortality Rate (%)', 'State_Region': ''},
    height=700,
    width=1000
)

fig_top_regions.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title='Average Mortality Rate (%)',
    font=dict(size=10)
)

fig_top_regions.write_html('hospital_mortality_regions_chart.html')
print("✓ Saved: hospital_mortality_regions_chart.html")

# Bible Belt focused chart
fig_bible_belt_regions = px.bar(
    bible_belt_sorted,
    x='Avg_Mortality_Rate',
    y='State_Region',
    orientation='h',
    color='Avg_Mortality_Rate',
    color_continuous_scale='YlOrRd',
    hover_data={'Hospital_Count': True},
    title='Bible Belt State Regions by Hospital Mortality Rate',
    labels={'Avg_Mortality_Rate': 'Mortality Rate (%)', 'State_Region': ''},
    height=600,
    width=1000
)

fig_bible_belt_regions.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title='Average Mortality Rate (%)',
    font=dict(size=10)
)

fig_bible_belt_regions.write_html('hospital_mortality_bible_belt_regions_chart.html')
print("✓ Saved: hospital_mortality_bible_belt_regions_chart.html")

# Regional comparison by state
state_summary = region_mortality.groupby('State').agg({
    'Avg_Mortality_Rate': 'mean',
    'Hospital_Count': 'sum'
}).reset_index().sort_values('Avg_Mortality_Rate', ascending=False)

print("\n" + "="*80)
print("AVERAGE MORTALITY BY STATE (across all regions)")
print("="*80)
print(f"\n{'State':<8} {'Rate':<10} {'Hospitals':<10}")
print("-"*40)
for idx, row in state_summary.head(15).iterrows():
    print(f"{row['State']:<8} {row['Avg_Mortality_Rate']:<10.2f} {int(row['Hospital_Count']):<10}")

print("\n" + "="*80)
print("STATE-REGION ANALYSIS COMPLETE")
print("="*80)
print("\nVisualizations created:")
print("  1. hospital_mortality_regions_chart.html - Top 25 regions nationally")
print("  2. hospital_mortality_bible_belt_regions_chart.html - Bible Belt regions")
print("\nThis finer-grained analysis shows geographic patterns WITHIN states,")
print("revealing which parts of each state struggle with hospital mortality.")
print("="*80)

fig_top_regions.show()
fig_bible_belt_regions.show()

Creating State-Region Analysis (North/South/East/West/Central)...

Mapped 3135 county coordinates

Total state-regions: 382

TOP 25 STATE REGIONS BY MORTALITY RATE

State-Region                   Rate       Hospitals 
--------------------------------------------------------------------------------
MS - Southeast                 5.62       7         
CO - Southeast                 5.25       2         
MS - Southwest                 5.17       6         
ND - North                     5.07       4         
MS - Northwest                 5.05       4         
NV - South                     5.00       1         
UT - Northeast                 5.00       1         
WA - Central                   5.00       1         
MS - West                      5.00       9         
MS - Northeast                 4.97       14        
AZ - Southwest                 4.95       2         
VT - Northeast                 4.95       2         
PA - Northeast                 4.94       8         
MT - Northea

In [15]:
import plotly.graph_objects as go
import plotly.express as px

print("Creating Geographic State-Region Bubble Map...\n")

# Calculate regional centroids from county coordinates
region_coords = {}

for state in county_mortality['State'].unique():
    state_data = county_mortality[county_mortality['State'] == state]
    
    for region in state_data['Region'].unique():
        region_data = state_data[state_data['Region'] == region]
        
        # Calculate center point of all counties in this region
        avg_lat = region_data['Latitude'].mean()
        avg_lon = region_data['Longitude'].mean()
        
        region_key = (state, region)
        region_coords[region_key] = (avg_lat, avg_lon)

print(f"Calculated centroids for {len(region_coords)} state-regions")

# Add coordinates to region_mortality
region_mortality['Latitude'] = region_mortality.apply(
    lambda row: region_coords.get((row['State'], row['Region']), (None, None))[0],
    axis=1
)
region_mortality['Longitude'] = region_mortality.apply(
    lambda row: region_coords.get((row['State'], row['Region']), (None, None))[1],
    axis=1
)

# Filter to regions with coordinates
region_with_coords = region_mortality[region_mortality['Latitude'].notna()].copy()

print(f"Regions with coordinates: {len(region_with_coords)}")

# Create bubble map
fig = px.scatter_geo(region_with_coords,
    lat='Latitude',
    lon='Longitude',
    size='Hospital_Count',
    color='Avg_Mortality_Rate',
    hover_name='State_Region',
    hover_data={
        'Avg_Mortality_Rate': ':.2f',
        'Hospital_Count': ':.0f'
    },
    size_max=80,
    color_continuous_scale='YlOrRd',
    scope='usa',
    title='US State Regions by Hospital Mortality Rate<br><sub>Bubble size = number of hospitals | Color = mortality rate (%)</sub>',
    labels={'Avg_Mortality_Rate': 'Mortality Rate (%)'}
)

fig.update_layout(
    height=850,
    width=1450,
    font=dict(size=11),
    geo=dict(
        projection_type='albers usa',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        showocean=True,
        oceancolor='rgb(204, 229, 255)',
        coastlinecolor='rgb(150, 150, 150)'
    ),
    coloraxis_colorbar=dict(
        thickness=15,
        len=0.7,
        x=1.02,
        title='Mortality<br>Rate (%)'
    ),
    hovermode='closest'
)

fig.write_html('hospital_mortality_state_regions_map.html')
print("\n" + "="*80)
print("✓ Map saved: hospital_mortality_state_regions_map.html")
print("="*80)
print("\nMap visualization shows:")
print("  - Each bubble = one state-region (N/S/E/W/Central)")
print("  - Bubble position = geographic center of that region")
print("  - Bubble size = number of hospitals in region")
print("  - Bubble color = mortality rate (yellow=low, red=high)")
print("  - Red clusters clearly show Bible Belt suffering zones")
print("="*80)

fig.show()

Creating Geographic State-Region Bubble Map...

Calculated centroids for 382 state-regions
Regions with coordinates: 382

✓ Map saved: hospital_mortality_state_regions_map.html

Map visualization shows:
  - Each bubble = one state-region (N/S/E/W/Central)
  - Bubble position = geographic center of that region
  - Bubble size = number of hospitals in region
  - Bubble color = mortality rate (yellow=low, red=high)
  - Red clusters clearly show Bible Belt suffering zones


In [ ]:
# Summary Analysis
print("=" * 70)
print("HOSPITAL MORTALITY RATE ANALYSIS SUMMARY")
print("=" * 70)
print(f"\nTotal Hospitals: {len(hybrid_hwm_clean)}")
print(f"Total States: {len(state_mortality)}")
print(f"Total ZIP Codes: {len(zip_mortality)}")

print(f"\nNational Average Mortality Rate: {hybrid_hwm_clean['Score_numeric'].mean():.2f}%")
print(f"National Median: {hybrid_hwm_clean['Score_numeric'].median():.2f}%")
print(f"Range: {hybrid_hwm_clean['Score_numeric'].min():.2f}% - {hybrid_hwm_clean['Score_numeric'].max():.2f}%")

print(f"\n" + "="*70)
print(f"TOP 15 ZIP CODES BY MORTALITY RATE")
print("="*70)
print(f"{'ZIP Code':<10} {'State':<8} {'Avg Rate':<12} {'Hospitals':<12} {'Hospital Name'}")
print("-" * 80)

top_zips = zip_mortality.nlargest(15, 'Avg_Mortality_Rate')
for idx, row in top_zips.iterrows():
    zip_code = row['ZIP Code']
    state_info = df[df['ZIP Code'].astype(str) == zip_code]['State'].iloc[0] if len(df[df['ZIP Code'].astype(str) == zip_code]) > 0 else 'N/A'
    print(f"{zip_code:<10} {state_info:<8} {row['Avg_Mortality_Rate']:<12.2f} {row['Hospital_Count']:<12} {row['Hospitals'][0][:35]}")